In [1]:
import random
import numpy as np
from deap import algorithms, base, creator, tools

#### Tekrarlanabilirlik için seed ayarlanır

In [2]:
SEED_VALUE = 32
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)

## Hiperparametreler

In [3]:
# Simüle edilecek nesil sayısı
n_generation = 32

# Popülasyon boyutu (birey sayısı)
n_population = 16

# Her bireyin gen sayısı
n_genes = 16

# Seçilim turnuvasında rastgele seçilecek birey sayısı (k)
selectionTournamentSize = 3

# Birey bazlı mutasyon olasılığı
individualMutationProbability = 0.05

# Gen bazlı mutasyon olasılığı
geneMutationProbability = 0.05

# Çaprazlama olasılığı
crossoverProbability = 0.90

## DEAP

#### "Uygunluk" ve "Birey" sınıflarını tanımla, uygunluğun bireye ait bir değer olacağını da tanımla

In [4]:
creator.create("Fitness", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

In [5]:
## Toolbox'ımız
toolbox = base.Toolbox()

#### Gen, birey ve popülasyon oluşturmamızı sağlayacak fonksiyonları oluşturup birbirine bağlayalım.

In [6]:
# Her bir bireyin her bir genini oluşturacak fonksiyonu toolbox içinde tanımla
# (0/1, boolean)
toolbox.register("initializeGene", random.randint, 0, 1)

# Bireyleri oluşturacak fonksiyonu toolbox içinde tanımla
toolbox.register("initializeIndividual", tools.initRepeat, creator.Individual, toolbox.initializeGene, n=n_genes)

# Popülasyonu oluşturacak fonksiyonu toolbox içinde tanımla
toolbox.register("initializePopulation", tools.initRepeat, list, toolbox.initializeIndividual, n=n_population)

*toolbox.initializePopulation* yoluyla bir popülasyon oluşturulurken, içinde bulunacak bireyler *toolbox.initializeIndividual* çağrılarak oluşturulur; her birey oluşturulurken ona ait olacak her bir gen ise *toolbox.initializeGene* çağrılarak tanımlanır.

#### Gen, birey ve popülasyon oluşturan toolbox fonksiyonlarımızı test edelim

In [7]:
print(
	"+ Örnek gen",
	toolbox.initializeGene()
)
print(
	"\n+ Örnek birey\n",
	np.array(toolbox.initializeIndividual())
)
print(
	"\n+ Örnek popülasyon (n=5)\n",
	np.array(toolbox.initializePopulation(n=5))
)

+ Örnek gen 0

+ Örnek birey
 [0 0 1 0 1 0 0 0 1 1 0 1 1 0 0 0]

+ Örnek popülasyon (n=5)
 [[1 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0]
 [1 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0]
 [1 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0]
 [1 0 0 0 1 0 1 1 1 1 0 1 0 0 1 1]
 [1 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0]]


#### Uygunluk fonksiyonumuzu toolbox yoluyla tanımlayalım (gen değerlerinin toplamı)

In [8]:
toolbox.register(
	"evaluate",
	lambda individual_genes: (sum(individual_genes), )
)

#### Çaprazlama, mutasyon ve seçilimi uygulayacak fonksiyonlarımızı toolbox yoluyla tanımlayalım

In [9]:
# Çaprazlama metodu: Tek noktalı çaprazlama
toolbox.register("mate", tools.cxOnePoint)

# Mutasyon metodu: Bit flip mutasyonu
toolbox.register("mutate", tools.mutFlipBit, indpb=geneMutationProbability)

# Seçilim metodu: Turnuva seçilimi, k = selectionTournamentSize
toolbox.register("select", tools.selTournament, tournsize=selectionTournamentSize)

#### Uygunluğu ölçen ve çaprazlama, mutasyon, seçilim gibi genetik operatörleri uygulayacak toolbox fonksiyonlarımızı test edelim

In [10]:
# Test için iki birey tanımlayalım
ind1 = toolbox.initializeIndividual()
ind2 = toolbox.initializeIndividual()

In [11]:
print(
	"+ Uygunluk skoru\n",
	np.array(ind1), toolbox.evaluate(ind1),
	"\n",
	np.array(ind2), toolbox.evaluate(ind2)
)

+ Uygunluk skoru
 [1 1 0 1 1 0 1 1 0 0 0 1 1 1 1 0] (10,) 
 [0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0] (5,)


In [12]:
print(
	"+ Tek Noktalı Çaprazlama\n",
	np.array(ind1), "Birey 1\n",
	np.array(ind2), "Birey 2\n",
	"\nYeni bireyler\n",
	np.array(toolbox.mate(ind1, ind2))
)

+ Tek Noktalı Çaprazlama
 [1 1 0 1 1 0 1 1 0 0 0 1 1 1 1 0] Birey 1
 [0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0] Birey 2
 
Yeni bireyler
 [[1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 0]
 [0 0 0 0 0 1 1 0 0 0 0 1 1 1 1 0]]


In [13]:
print(
	"+ Bit Flip Mutasyonu\n",
	np.array(ind1), "Birey\n",
	np.array(toolbox.mutate(ind1)[0]), "Mutasyon uygulanmış hali",
)

+ Bit Flip Mutasyonu
 [1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 0] Birey
 [1 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0] Mutasyon uygulanmış hali


In [14]:
print("+ Turnuva Seçilimi (parent selection) (k=2)")
print("\nPopülasyon:")

# Örnek popülasyon tanımla
examplePopulation = toolbox.initializePopulation(n=6)

# Bireylerin uygunluklarını hesaplayıp üstlerine yaz (seçilim operatörü bu değeri okuyarak seçiyor)
fitnesses = [toolbox.evaluate(ind) for ind in examplePopulation]
for ind, fitnessScore in zip(examplePopulation, fitnesses):
	ind.fitness.values = fitnessScore

# Popülasyonu uygunluk skorlarıyla yazdır
for ind in examplePopulation:
	print(np.array(ind), ind.fitness.values[0])

# Seçilim operatörünü uygula
# tournsize, daha önce gösterdiğimiz k parametresi, yani uygunluğa bakılmadan kaç birey seçilip aralarında turnuva yapılacağını belirler
# Burada geçen k parametresi ise turnuvanın kaç defa gerçekleşeceği
selectedInd = toolbox.select(examplePopulation, k=1, tournsize=2)[0]
print("\nSeçilen birey: ", selectedInd, toolbox.evaluate(selectedInd)[0])

+ Turnuva Seçilimi (parent selection) (k=2)

Popülasyon:
[0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0] 4.0
[0 0 0 1 1 0 0 1 1 0 0 1 0 1 0 0] 6.0
[1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 0] 10.0
[0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1] 6.0
[1 1 0 1 1 1 0 1 1 0 1 1 0 0 0 1] 10.0
[0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 1] 8.0

Seçilen birey:  [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1] 6


## Simülasyonu çalıştırma zamanı!

In [15]:
print("--- Simülasyon zamanı! ---")

# Başlangıç popülasyonunu oluştur
initialPopulation = toolbox.initializePopulation(n=n_population)

# Simülasyon boyunca en uygun bireyin kaydını tutacak objemiz
hallOfFame = tools.HallOfFame(1)

# Uygunluk istatistiklerini tutacak objemiz
stats = tools.Statistics(lambda ind: ind.fitness.values[0])
stats.register("min", np.min)
stats.register("max", np.max)

print("+ Başlangıç popülasyonu:")
print(np.array(initialPopulation))

--- Simülasyon zamanı! ---
+ Başlangıç popülasyonu:
[[0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1]
 [0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 1]
 [0 0 0 1 0 1 0 0 1 1 0 1 0 1 0 0]
 [1 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0]
 [1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1]
 [1 0 0 1 1 0 1 1 1 0 0 1 1 1 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1]
 [1 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0]
 [0 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1]
 [0 1 1 0 0 0 1 0 0 1 1 1 0 0 0 0]
 [1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1]
 [0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 1]]


_They look like my high school grades_

In [16]:
# Simülasyon başlasın!
finalPopulation, logs = algorithms.eaSimple(
	population=initialPopulation,
	toolbox=toolbox,
	halloffame=hallOfFame,
	stats=stats,
	ngen=n_generation,
	cxpb=crossoverProbability,
	mutpb=individualMutationProbability,
	verbose=True
)

gen	nevals	min	max
0  	16    	2  	11 
1  	14    	5  	11 
2  	16    	8  	13 
3  	12    	9  	14 
4  	14    	11 	15 
5  	12    	12 	14 
6  	16    	13 	14 
7  	8     	14 	14 
8  	15    	13 	15 
9  	14    	14 	15 
10 	13    	13 	15 
11 	16    	15 	15 
12 	11    	14 	15 
13 	12    	14 	15 
14 	14    	14 	15 
15 	14    	15 	15 
16 	14    	14 	16 
17 	16    	15 	16 
18 	14    	15 	16 
19 	14    	15 	16 
20 	12    	14 	16 
21 	14    	16 	16 
22 	14    	14 	16 
23 	16    	16 	16 
24 	16    	15 	16 
25 	12    	16 	16 
26 	14    	16 	16 
27 	16    	16 	16 
28 	16    	16 	16 
29 	16    	16 	16 
30 	12    	16 	16 
31 	14    	16 	16 
32 	12    	16 	16 


In [17]:
bestIndividual = hallOfFame[0]
print("+ En iyi uygunluk sağlayan bireyin skoru: {}".format(bestIndividual.fitness.values[0]))
print("Genotipi: ", bestIndividual)

+ En iyi uygunluk sağlayan bireyin skoru: 16.0
Genotipi:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [18]:
print("+ Final popülasyon:")
print(np.array(finalPopulation))

+ Final popülasyon:
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


##### Gen sayısı çok küçük ve problem de çok basit olduğu için, optimal çözümün bulunması hiç zor olmadı doğal olarak :)